# What to expect

In notebooks 2A and 2B we took a first look at the results from STAR, loading them as a DeseqDataSet and normalizing them with DeSeq2. In this session we are going to look at which genes are the most highly differentially expressed, and investigate the GO terms and pathways associated with them. We will do this first for the example dataset <i>Schistosoma mansoni</i>, and then in notebook 3B you will repeat the process for your chosen dataset.

In [ ]:
#Install PyDESeq2 and import required classes
! pip install --quiet pydeseq2

In [ ]:
import pandas as pd
from pydeseq2.dds import DeseqDataSet

# load in the counts and metadata again
counts = pd.read_csv("analysis/Schistosoma_mansoni/star/ReadsPerGene.csv", index_col=0).T
metadata = pd.read_csv("data/Schistosoma_mansoni/metadata.csv", index_col=0)

# restrict to the 2 stages we want to compare
counts_s = counts[metadata["stage"].isin(["cercarium","24 hr schistosomulum"])]
metadata_s = metadata[metadata["stage"].isin(["cercarium","24 hr schistosomulum"])]

# create deseq2 dataset object
dds = DeseqDataSet(
    counts=counts_s,
    metadata=metadata_s,
    design_factors="stage",  # compare samples based on the developmental "stage"
    refit_cooks=True
)

# Differential Expression analysis

As we did in notebook 2A, we will apply the `deseq2` method to our dds object. Remember that this method normalises the data, estimates the dispersion and calculates the log fold change (LFC) based on the design factor.

In [ ]:
# Run DeSeq2
dds.deseq2()

This time, however, we also want to perform a statistical analysis, to find out which differences in gene expression are statistically significant. To do that, we use the class `DeseqStats` on our dds object, and store the output in a new object called "stat_res" 

In [ ]:
from pydeseq2.ds import DeseqStats

stat_res=DeseqStats(dds)

Now, we have to generate a summary of the statistical analysis contained in the "stat_res" object. To do that, we use the `summary` method.

In [ ]:
stat_res.summary()

Voila! This is the raw result of our differential expression analysis. We can find out more about the columns in these results on the [page for the DeSeq2 gene-level differential expression workflow](https://master.bioconductor.org/packages/release/workflows/vignettes/rnaseqGene/inst/doc/rnaseqGene.html), as the python module we are using is a wrapper for the original tool written in R. Let's take a moment to make sure we understand it. 

<div class="alert alert-block alert-warning">

Questions:
    
1. baseMean: what is this value? How is it calculated?
    
2. log2FoldChange: why do you think the fold change is calculated as log2? 
    For gene Smp318880, the log2FoldChange is 4.93. Is this gene more expressed in cercarium or in 24 hr schistosomulum? How many times more?
    
3. lfcSE: what is this value?
    
4. stat: what is this value?
    
5. pvalue and padj: what is the difference between these two values?

<div class="alert alert-block alert-success">

Answers:



We will now store the results in a dataframe, so we can explore the results further. It is also a good idea to save the dataframe as a csv file, in case we want to use outside Noteable.

In [ ]:
import numpy as np

# use ".results_df" to store results in a dataframe called "res"


# replace p-values of 0 with a very small number as otherwise they cause errors
res.loc[ res.pvalue == 0, "pvalue" ] = np.finfo(np.float64).tiny
res.loc[ res.padj == 0, "padj" ] = np.finfo(np.float64).tiny

# save to csv file
res.to_csv(f"analysis/Schistosoma_mansoni/cercarum_vs_24h_schistosomulum.full.csv")

You probably saw in the stat_res summary that some genes have very low expression, indicated by a low baseMean. The results for a gene that is very lowly expressed will probably not be very reliable, or very informative of a biological function. They might also make further visualisation skewed. To avoid this, we will filter results to remove genes with very low expression. A common (although arbitrary) threshold for this is a baseMean of 10. 

<div class="alert alert-block alert-warning">

Task:

6. Filter out results with baseMean<10

In [ ]:
# Filter results with baseMean<10 so that gene expressions close to zero don't skew results


Lets start exploring the results. 

<div class="alert alert-block alert-warning">

Questions: 

7. How many genes are significantly differentially expressed?
8. For how many of these significant genes is the fold change (FC) greater than 2 or less than 0.5? Store them in a new dataframe called "sigs", and save it as a csv file

<div class="alert alert-block alert-success">

Answers: 



In [ ]:
# Count the number of genes with padj<0.05


In [ ]:
# Get list of only genes that have a fold change FC > 2 or FC < 0.5 and are significantly changed


In [ ]:
# save it as a csv file
sigs.to_csv(f"analysis/Schistosoma_mansoni/cercarum_vs_24h_schistosomulum.filtered.csv")

# Visualisation - Volcano Plot

<figure>
    <img src="https://scienceparkstudygroup.github.io/rna-seq-lesson/img/volcano_plot.png" align="right" width="400">
</figure>

Another common way to visualise differential expression analysis results is a volcano plot.
The advantage of this plot is that we can see at the same time the change in gene expression and the statistical significance of that change, for each gene. That makes it easy to identify genes for further analysis.
As shown in the figure on the right (taken from [this source](https://scienceparkstudygroup.github.io/rna-seq-lesson/06-differential-analysis/index.html#3-volcano-plot)), in a typical volcano plot:
* on the X axys we plot the log2 fold change in gene expression
* on the Y axys we plot the -log10Pvalue. Why? Because:

    (a) The p-value is transformed into Log10 to help with visualisation, in the same way that the fold change in gene expression was calculated as log2 by PyDeseq2. 

    (b) If you remember previous stats lessons, P values are a probability, and therefore are always between 0 and 1. The log10 of a number between 0 and 1 will always be negative. To make interpretation easier, we do "-"log10, so the result is always a positive number. For example: log10(0.5)=-0.301 ; -[log10(0.5)]=-[-0.301]=0.301.

Let's plot our results in a volcano plot. To do that, we will use matplotlib, which we importas "plt", and which you already used in the Data Exploration last year. If you need a quick refresher, have a look [here](https://matplotlib.org/stable/users/explain/quick_start.html)

<div class="alert alert-block alert-warning">
    
9. Using matplotlib, make a scatter plot that: 
* in the X axys plots the log2FoldChange values from the "res" dataframe 
* in the Y axys plots -log10 of the padj values from the "res" dataframe

<details>
<summary><i>Hint</i></summary>

- `apply` a lambda function to transform the padj to log10 using a numpy
- you can have a look at the Data Exploration in Biology class 3 (week 2), where lambda functions were introduced
    
</details>

In [ ]:
import seaborn as sns
import matplotlib.pylab as plt



Let's make the volcano plot fancier by coloring dots depending on:
* whether they are up- or down- regulated
* whether they are significantly regulated

In [ ]:
# define the significantly up or down regulated genes
down = res[(res['log2FoldChange']<=-1)&(res['padj']<0.05)]
up = res[(res['log2FoldChange']>=1)&(res['padj']<0.05)]

# plot the all the genes and then highlight downregulated and upregulated


Let's improve the volcano plot further by adding:
* lines at the threshold values
* axys labels

And let's save it as a png image

In [ ]:
# define the significantly up or down regulated genes
down = res[(res['log2FoldChange']<=-1)&(res['padj']<0.05)]
up = res[(res['log2FoldChange']>=1)&(res['padj']<0.05)]

# plot the all the genes and then highlight downregulated and upregulated

# Add axys labels

# Add threshold lines
plt.axvline(-1,color="grey",linestyle="--")
plt.axvline(1,color="grey",linestyle="--")
plt.axhline(-np.log10(0.05),color="grey",linestyle="--")
plt.legend()

plt.savefig('example_volcano.png')

# Visualisation - Interactive Volcano Plot (Extension)

To explore our data, it would be very convenient if we coud hover over a dot and get the gene name. Making that sort of interactive plot is possible using the python library [plotly](https://plotly.com/python/).

In [ ]:
# First, we install and import
! pip install plotly
! pip install chart_studio
import plotly.express as px
import chart_studio.plotly as py

In [ ]:
# Plotly requires a slightly different input than matplotlib. In our dataframe res, we need to:
# - create a new column with the gene names
# - create a new column with the transformed -log10(padj)
res['gene']=res.index
res['-log10 p value']=-np.log10(res['padj'])

In [ ]:
# lets check that it worked
res

In [ ]:
# Now, we can use our new columns to make the volcano plot
fig = px.scatter(
    res,
    x='log2FoldChange',
    y='-log10 p value',
    hover_data=['log2FoldChange', 'padj','gene'],
    title='Volcano Plot cercarum_vs_24h_schistosomulum'
)

# Show the plot
fig.show() # this takes a while to appear

# Visualisation - Clustermap (Extension)

<figure>
    <img src="https://scienceparkstudygroup.github.io/rna-seq-lesson/img/06-basic-heatmap.png" align="right" width="400">
</figure>

A common way to visualise gene expression is a [clustermap](https://seaborn.pydata.org/generated/seaborn.clustermap.html#seaborn.clustermap). 

Clustermaps combine a heatmap, which you already used in Data Exploration (class 5, week 3 and others), with clustering. As shown in the figure on the right (taken from [this source](https://scienceparkstudygroup.github.io/rna-seq-lesson/06-differential-analysis/index.html#4-heatmap), gene expression values are represented on a color scale, and genes with a similar pattern of expression are grouped together.

To improve visualisation, gene expression values are transformed to logarithm before graphing them. Because some genes might have expression=0, and the log of 0 is undefined, often the `log1p` transformation is applied. This returns the logarithm of (1 + expression value). So, if the expression of a particular gene is 0, instead of returning an error (because log(0) = undefined), it will return 0 (because log(1+0)=0). 

Lets visualise our significantly regulated genes, which we have stored in the sigs dataframe, in a clustermap.

In [ ]:
# perform the log transformation on the normalized counts stored in dds.layers['normed_counts']
# and save the result in a new layer called log1p
dds.layers['log1p'] = np.log1p(dds.layers['normed_counts'])

# from our dds, select only the genes that are significantly regulated (which we stored in the sigs dataset)
# and save the result as a new dds object called dds_sigs
dds_sigs = dds[:, sigs.index]

# create a dataframe called grapher that 
# - contains the log1p layer we just created, but transposed so that genes are rows and samples are columns
# - as index has the gene names from dds_sigs
# - as column names has the sample names from dds_sigs
grapher = pd.DataFrame(dds_sigs.layers['log1p'].T,
                       index=dds_sigs.var_names, columns=dds_sigs.obs_names)
# make a clustermap of the data in the grapher dataframe
sns.clustermap(grapher, z_score=0, cmap = 'RdYlBu_r')

<div class="alert alert-block alert-warning">
    
Question:

10. Looking at the heatmap, would you say that:
    
    a. The majority of genes are upregulated in cercarium compared to 24h schistosomulum?
   
    b. The majority of downregulated in cercarium compared to 24h schistosomulum?
   
    c. Roughly half of the genes are upregulated in half are downregulated in cercarium compared to 24h schistosomulum?
    

<div class="alert alert-block alert-success">
    
Answers:

